In [ ]:
!pip install torch
!pip install -e git+https://github.com/OpenAccess-AI-Collective/axolotl#egg=axolotl
!pip install flash-attn
!pip install deepspeed
!pip install mlflow

In [7]:
import yaml

# Your YAML string
yaml_string = """
base_model: meta-llama/Meta-Llama-3-8B-Instruct
model_type: AutoModelForCausalLM
tokenizer_type: AutoTokenizer

load_in_8bit: false
load_in_4bit: true
strict: false

datasets:
  - path: medalpaca/medical_meadow_medqa
    type: alpaca
dataset_prepared_path:
val_set_size: 0.1
output_dir: ./outputs

adapter: qlora
lora_model_dir:

sequence_len: 4096
sample_packing: true
pad_to_sequence_len: true

lora_r: 32
lora_alpha: 16
lora_dropout: 0.05
lora_target_modules:
    - layers.26.self_attn.v_proj
    - layers.17.self_attn.v_proj
    - layers.28.self_attn.v_proj
    - layers.3.self_attn.v_proj
    - layers.29.self_attn.v_proj
    - layers.21.self_attn.v_proj
    - layers.4.mlp.up_proj
    - layers.0.mlp.up_proj
    - layers.1.mlp.down_proj
    - layers.3.mlp.up_proj
    - layers.16.self_attn.v_proj
    - layers.15.self_attn.v_proj
    - layers.0.mlp.down_proj
    - layers.20.self_attn.v_proj
    - layers.25.self_attn.v_proj
    - layers.5.mlp.up_proj
    - layers.7.mlp.up_proj
lora_target_linear: true
lora_fan_in_fan_out:

wandb_project: medsft
wandb_entity:
wandb_watch:
wandb_name:
wandb_log_model:

gradient_accumulation_steps: 8
micro_batch_size: 2
num_epochs: 2
optimizer: paged_adamw_32bit
lr_scheduler: cosine
learning_rate: 0.00002

train_on_inputs: false
group_by_length: false
bf16: auto
fp16:
tf32: false

gradient_checkpointing: true
early_stopping_patience:
resume_from_checkpoint:
local_rank:
logging_steps: 1
xformers_attention:
flash_attention: true

warmup_steps: 10
evals_per_epoch: 2
eval_table_size:
saves_per_epoch:
save_strategy:
save_steps:
debug:
deepspeed: deepspeed_configs/zero2.json
weight_decay: 0.0
fsdp:
fsdp_config:
special_tokens:
  pad_token: "<|end_of_text|>"
"""

# Convert the YAML string to a Python dictionary
yaml_dict = yaml.safe_load(yaml_string)

# Specify your file path
file_path = 'multigpu.yaml'

# Write the YAML file
with open(file_path, 'w') as file:
    yaml.dump(yaml_dict, file)

**Axolotl command for starting training**

In [ ]:
!accelerate launch -m axolotl.cli.train multigpu.yaml